# Pyro experiments

In [7]:
import numpyro
import torch
import pyro

pyro.set_rng_seed(101)

## Inference
- exact bayesian inference - often intractable
- Markov Monte Carlo - computationally expensive
- Variatinal Inference 

### Metropolis sampling

sampling rule satisfies the Markov property. Every observation is not depend on observation before (memory-less).

$$
\frac{P(A|b_1) * p(b_1)}{P(A|b_2) * p(b_2)} = \frac{ \frac{ P(A|b_1) * p(b_1) } { p(A) } }{ \frac { P(A|b_2) * p(b_2) } { p(A) } }
$$

improovements
- Hamiltonian Monte Carlo (HMC), 1987
- Intelligently set parameters (step-size, number of steps) to HMC
  - No U_Turn Sampling (NUTS), 2011
  - Matthew Hoffman, Andrew Gelman
  
  
## Where does probabillistic programming work good
treating uncertainty as a first-class citizen and transparently specifying any assumptions

- Model validation when test data is expensive
  - train model on general and readily-available dataset (source dataset) and then test on target (domain-specific) data (assumably without labels)
  - live model validation on random new data
- Systems that make use of export knowledge
  - What is the correct structure for a graphical model? What influences what?
  - What are reasonable priors for unobserved parameters?
 
- Problems that require reasoning with uncertainty
  - Should we make this changes to our process?
  - If we make this change, what is the worst-case outcome? Best-case? How sure are we that the situation will improve?

In [5]:
!pip install pyro-ppl[extras]

     |████████████████████████████████| 573 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.9 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 2.9 MB/s eta 0:00:01
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=bef31c2dde9570372660281ff4a57a0c68f454af1a7edca15052f3acbf338286
  Stored in directory: /home/eugene/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
ERROR: torchvision 0.7.0 has requirement torch==1.6.0, but you'll have torch 1.5.0 which is incompatible.
ERROR: pytorch-toolbelt 0.3.2 has requirement Pillow<7.0,>=6.0, but you'll have pillow 7.1.2 which is incompatible.
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.6.0a0+b68adcf
    Uninstalling torchvision-0.6.0a0+b68adcf:
      Successfully uninstalled torchvision-0.6.0a0+b68adcf


In [6]:
torch.__version__

'1.5.0'

## Primitives
Draw a sample

In [18]:
normal = torch.distributions.Normal(0, 1.0)
x = normal.rsample(sample_shape=(10,))
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor([-1.0760, -1.3107, -0.1720, -1.8083, -0.4594, -0.5810,  0.4576, -0.5299,
        -0.7282,  1.2456])
log prob tensor([-1.4978, -1.7780, -0.9337, -2.5539, -1.0245, -1.0877, -1.0236, -1.0593,
        -1.1840, -1.6946])


probabalistic model

In [25]:
def weather_pytorch():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()


def weather_pyro():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print('pytorch:', weather_pytorch())
    print('pyro:', weather_pyro())

pytorch: ('sunny', 87.1436996459961)
pyro: ('sunny', 64.85323333740234)
pytorch: ('cloudy', 46.7928352355957)
pyro: ('sunny', 92.76007080078125)
pytorch: ('sunny', 69.12490844726562)
pyro: ('sunny', 50.834327697753906)


In [40]:
def ice_cream_sales(weather):
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

for _ in range(3):
    print('ice_cream_sales (pyro):', ice_cream_sales(weather_pyro))
    print('ice_cream_sales (pytorch):', ice_cream_sales(weather_pytorch))    

ice_cream_sales (pyro): tensor(61.2207)
ice_cream_sales (pytorch): tensor(192.9776)
ice_cream_sales (pyro): tensor(60.7514)
ice_cream_sales (pytorch): tensor(57.5971)
ice_cream_sales (pyro): tensor(47.5173)
ice_cream_sales (pytorch): tensor(51.7640)


In [39]:
def geometric(p, t=None):
    """
    calculate the number of failures until the first success
    """
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

0


### Coin flipping problem

In [9]:
TRIALS = [0,1,1,0,1,1,1,0,1,0,1,1]
with pm.Model() as success_model:
    p_success = pm.Uniform('p_success', lower=0, upper=1)
    trials = pm.Bernoulli('trials', p=theta_one, oversved=TRIALS)
    
mean_success = trace_df.p_success.mean()
lower_credibility = trace_df.p_success.quantile(q=0.05)
higher_credibility = trace_df.p_success.quantile(q=0.95)
print(f'Mean accuracy estimate = {mean_success}')
print(f'90% credibility interval = {lower_credibility} to {higher_credibility}')

NameError: name 'pm' is not defined

In [ ]:
a